In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import dataset
import utils
assert tf.config.list_physical_devices('GPU')

Using TensorFlow backend.


In [3]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(EMBED_DIM=110, UNITS=220):

    layer = input_text = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)
    layer = layers.add([layer,
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer))
    ]
    model = tf.keras.Model(inputs=[input_text], outputs=outputs)

    # tf.keras.utils.plot_model(model, to_file='model.png')
    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 110)      4840        input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 220)      582560      embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 220)      776160      bidirectional[0][0]              
______________________________________________________________________________________________

In [3]:
def fit(data, scheduler, verbose=1):
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, utils.CircularLearningRate):
        scheduler.set_dataset(data, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
    x  = data.normalized_texts
    vx = data.normalized_validation
    y  = {'N': data.niqqud_texts,      'D': data.dagesh_texts,      'S': data.sin_texts,      'C': data.normalized_texts     }
    vy = {'N': data.niqqud_validation, 'D': data.dagesh_validation, 'S': data.sin_validation, 'C': data.normalized_validation}
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    return dataset.load_file(filenames, BATCH_SIZE, validation, maxlen=maxlen, shuffle=True)

In [5]:
data_rabanit = load_data(['birkat_hamazon.txt', 'kuzari.txt', 'hakdama_leorot.txt', 'hartzaat_harav.txt', 'orhot_hayim.txt', 'rambam_mamre.txt', 'short_table.txt', 'tomer_dvora.txt'])

In [6]:
data_pre_modern = load_data(['elef_layla.txt', 'bialik', 'shaul_tchernichovsky', 'breslev.txt', 'itzhak_berkman', 'zevi_scharfstein', 'pesah_kaplan', 'abraham_regelson',
                             'elisha_porat', 'uriel_ofek', 'yisrael_dushman', 'zvi_zviri', 'atar_hashabat.txt', 'ali_baba.txt'])

In [7]:
data_modern = load_data(validation=0.2, source=['forums', 'newspapers', 'wiki', 'blogs', 'adamtsair.txt', 'katarsis.txt'])  # , 'imagination.txt', 'sipurim.txt', 'ricky.txt'

In [10]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, utils.CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

Train on 76377 samples, validate on 8487 samples
76377/76377 [==============================] - 92s 1ms/sample - loss: 0.3310 - N_loss: 0.2578 - D_loss: 0.0634 - S_loss: 0.0097 - N_accuracy: 0.9099 - D_accuracy: 0.9759 - S_accuracy: 0.9975 - val_loss: 0.1308 - val_N_loss: 0.0980 - val_D_loss: 0.0302 - val_S_loss: 0.0024 - val_N_accuracy: 0.9672 - val_D_accuracy: 0.9893 - val_S_accuracy: 0.9994


In [11]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, utils.CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

Train on 84096 samples, validate on 9344 samples
84096/84096 [==============================] - 92s 1ms/sample - loss: 0.2547 - N_loss: 0.1918 - D_loss: 0.0570 - S_loss: 0.0059 - N_accuracy: 0.9356 - D_accuracy: 0.9771 - S_accuracy: 0.9984 - val_loss: 0.1905 - val_N_loss: 0.1396 - val_D_loss: 0.0461 - val_S_loss: 0.0047 - val_N_accuracy: 0.9537 - val_D_accuracy: 0.9817 - val_S_accuracy: 0.9988ss: 0.2567 - N_loss: 0.1934 - D_loss: 0.0573 - S_loss: 0.0060 - N_accuracy: 0.9350 - D_accuracy:  - ETA: 0s - loss: 0.2553 - N_loss: 0.1923 - D_loss: 0.0571 - S_loss: 0.0059 - N_accuracy: 0.9354 - D_accuracy: 0.9771 - S_acc


In [12]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, utils.CircularLearningRate(6e-3, 6e-3, 0.5e-3))

Train on 12236 samples, validate on 3060 samples
12236/12236 [==============================] - 24s 2ms/sample - loss: 0.2321 - N_loss: 0.1765 - D_loss: 0.0502 - S_loss: 0.0056 - N_accuracy: 0.9461 - D_accuracy: 0.9812 - S_accuracy: 0.9988 - val_loss: 0.1733 - val_N_loss: 0.1296 - val_D_loss: 0.0390 - val_S_loss: 0.0044 - val_N_accuracy: 0.9614 - val_D_accuracy: 0.9859 - val_S_accuracy: 0.9990


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

In [ ]:
def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized_validation[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud_validation[s], data.dagesh_validation[s], data.sin_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(data_modern, 0)

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
input = layers.Input((2, 1))
layer = layers.Bidirectional(layers.GRU(units=50, return_sequences=True))(input)
output = layers.Dense(1, activation='sigmoid')(layer)
model = tf.keras.Model(inputs=[input], outputs=[output])

lr = 1
for i in range(4):
    lr /= 3
    x = [[[np.random.random()], [np.random.random()]] for _ in range(100000)]
    y = [[[a], [a]] for [[a], [_]] in x]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mean_squared_error")
    model.fit(x, y, epochs=1, verbose=1)
print(model.predict([[[1], [0.5]]]))
print(model.evaluate([[[1], [0.5]]], [[[1], [1]]]))
# print(result)